### Managing the Coversation History

One important concept to understand when building chatbots is how to manage conversation histroy. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

"trim_messages" help to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we wnat to always keep the system message and whether to allow partial messages.

In [1]:
import os
from langchain_groq import ChatGroq
import dotenv

dotenv.load_dotenv()

groq_api_key = os.getenv("GROQ_API")

model = ChatGroq(model="openai/gpt-oss-120b", api_key=groq_api_key)

c:\Users\pouru\Desktop\LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from langchain_core.messages import HumanMessage, trim_messages, SystemMessage, AIMessage

trimmer = trim_messages(
    max_tokens=500,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="hello, I'm Pourush Gupta and I am one of the greatest AI engineer from India"),
    AIMessage(content="Hello Pourush, How can I help you today?"),
    HumanMessage(content="Can you remind me my name?"),
    AIMessage(content="Your name is Pourush Gupta."),
    HumanMessage(content="What is my profession?"),
    AIMessage(content="You are an AI engineer."),
    HumanMessage(content="Which country do I live in?"),
    AIMessage(content="You live in India."),
]

trimmer.invoke(messages)


[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hello, I'm Pourush Gupta and I am one of the greatest AI engineer from India", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Pourush, How can I help you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Can you remind me my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Pourush Gupta.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my profession?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='You are an AI engineer.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Which country do I live in?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='You live in India.', additional_kwargs={}, response_metadata={})]

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assitant and you need to answer the user query in the {language} language"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [16]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What were we talking about?")],
        "language":"English"
    }
)

print(response.content)

We’ve been talking about you—your name (Pourush Gupta), your profession (an AI engineer), and the fact that you’re based in India. Let me know if there’s anything else you’d like to discuss!


In [17]:
# Wrapping this into message history

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

session_store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable":{"session_id":"chat1"}}

response = with_message_history.invoke(
    {
        "messages":messages + [HumanMessage(content="what's my name?")],
        "language":"english",
    },
    
    config=config
)

print(response.content)




Your name is Pourush Gupta.
